# European Soccer Dataset – Player Contract Value Analysis

**Goal:** "Do Something! NO WINE DATA" -Seth. I want to identify which player attributes are the most important drivers of a player's overall rating, which I will use as a proxy for player contract value.

I am going touse the [European Soccer Database](https://www.kaggle.com/datasets/hugomathien/soccer) from Kaggle, which contains FIFA-sourced player attributes across 11 European leagues (2008–2016).

Load in the dataset and the packages 

Using SQLlite to query the database and avoid loading the entire dataset into my poor laptops memory

In [ ]:

import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

sns.set_theme(style='whitegrid', palette='viridis')
plt.rcParams['figure.figsize'] = (12, 6)

#Connect to the SQLite database (I want to clean the data somewhere outside of my laptop)
conn = sqlite3.connect('../data/database.sqlite')
print('Connected to database.')

## 1. Data Overview

List all tables and inspect row counts.

In [ ]:

tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
print('Tables in database:')
for t in tables['name']:
    count = pd.read_sql(f'SELECT COUNT(*) as cnt FROM [{t}]', conn)['cnt'][0]
    print(f'  {t}: {count:,} rows')

Bring in the needed tables and connect to individual players

In [ ]:

players = pd.read_sql('SELECT * FROM Player', conn)
player_attrs = pd.read_sql('SELECT * FROM Player_Attributes', conn)

print(f'Players: {players.shape}')
print(f'Player Attributes: {player_attrs.shape}')
player_attrs.head()

## 2. Player Attributes – Schema & Missing Values

Load in the dataframe and find null values.

In [ ]:

info = pd.DataFrame({
    'dtype': player_attrs.dtypes,
    'non_null': player_attrs.count(),
    'null_pct': (player_attrs.isnull().sum() / len(player_attrs) * 100).round(1)
})
info

## 3. Prepare Feature Matrix

We select the numeric FIFA attribute columns (everything except IDs, dates, and `preferred_foot` / `attacking_work_rate` / `defensive_work_rate`) and use `overall_rating` as our target variable — a proxy for contract value.

In [ ]:
#Columns to exclude (non-feature columns)
exclude_cols = [
    'id', 'player_fifa_api_id', 'player_api_id', 'date',
    'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'
]

# Target variable
target = 'overall_rating'

# Feature columns: all numeric columns except excluded ones and the target
feature_cols = [
    c for c in player_attrs.columns
    if c not in exclude_cols and c != target
    and player_attrs[c].dtype in ['int64', 'float64']
]

print(f'Target: {target}')
print(f'Features ({len(feature_cols)}): {feature_cols}')

Drop null values from dataset

In [ ]:

dataset = player_attrs[feature_cols + [target]].dropna()
print(f'Clean rows: {len(dataset):,} (dropped {len(player_attrs) - len(dataset):,} with NaNs)')

X = dataset[feature_cols]
y = dataset[target]

## 4. Correlation Heatmap

Visualise how the features correlate with each other and with `overall_rating`.

In [ ]:

corr_with_target = X.corrwith(y).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(10, 8))
corr_with_target.plot.barh(ax=ax, color=sns.color_palette('viridis', len(corr_with_target)))
ax.set_xlabel('Pearson Correlation with Overall Rating')
ax.set_title('Feature Correlation with Overall Rating (Contract Value Proxy)')
ax.axvline(0, color='gray', linewidth=0.8)
plt.tight_layout()
plt.show()

## 5. Random Forest Feature Importance

Train a Random Forest regressor to predict `overall_rating` from all attributes, then extract feature importances to determine what matters most for player value.

Run a RandomForestRegressor and then evaluate the model 

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


rf = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)


y_pred = rf.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'Random Forest Performance:')
print(f'  R² Score: {r2:.4f}')
print(f'  MAE:      {mae:.4f}')

Find the best features and plot the results

In [ ]:

importances = pd.Series(rf.feature_importances_, index=feature_cols)
importances = importances.sort_values(ascending=True)

fig, ax = plt.subplots(figsize=(10, 10))
importances.plot.barh(
    ax=ax,
    color=sns.color_palette('magma', len(importances))
)
ax.set_xlabel('Feature Importance (Gini)')
ax.set_title('What Drives Player Contract Value?\n(Random Forest Feature Importance)')
plt.tight_layout()
plt.show()

What are the top 10 most important attributes to get a contract?

## 6. Top 10 Most Important Attributes

In [ ]:
top10 = importances.tail(10).sort_values(ascending=False)

print('=== TOP 10 MOST IMPORTANT PLAYER ATTRIBUTES ===')
print('  (for predicting overall rating / contract value)\n')
for rank, (attr, imp) in enumerate(top10.items(), 1):
    bar = '█' * int(imp * 200)
    print(f'  {rank:2d}. {attr:<25s}  {imp:.4f}  {bar}')

Show the top 4 so it is less crowded

In [ ]:

top4 = top10.index[:4].tolist()

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
for ax, feat in zip(axes.ravel(), top4):
    ax.scatter(df[feat], df[target], alpha=0.05, s=5, color='steelblue')
    ax.set_xlabel(feat)
    ax.set_ylabel('Overall Rating')
    ax.set_title(f'{feat} vs Overall Rating')

plt.suptitle('Top 4 Contract Value Drivers', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Summary

The analysis above identifies the player attributes that matter most for determining a player's overall rating — our stand-in for contract value. Key takeaways:

- **Reactions, potential, and short passing** tend to rank among the top attributes driving overall rating.
- The Random Forest model achieves a high R² score, confirming that FIFA attributes collectively explain most of the variation in overall rating.
- The feature importance chart provides a clear ranking you can use to prioritise in contract negotiations.

### Next steps
- Incorporate transfer fee / wage data from external sources to use actual contract values instead of the rating proxy.
- Break down importance by playing position (GK vs. defender vs. forward).
- Time-series analysis: how attribute importance shifts across seasons.

In [ ]:
conn.close()
print('Done! Database connection closed.')